In [221]:
import pandas as pd
from datetime import datetime
import re
pd.options.mode.chained_assignment = None

In [222]:
df = pd.read_excel('flight_info.xlsx')

In [223]:
df = df[['序号', '1、姓名', '2、华盛顿大学邮箱(UW_NETID@uw.edu)',
         '3、出发城市：', '4、出发日期：', '5、航班出发的当地时间：（请用24小时制填写，例如：17:30）', 
         '6、落地西雅图的日期：', '7、航班抵达西雅图的当地时间：（请用24小时制填写，例如：17:30）', 
         '8、抵达西雅图航班的航班号：']]

In [224]:
df.columns = ['form_id', 'name','email', 'dep_city', 'dep_date',
              'dep_time', 'arr_date', 'arr_time', 'flight_num']

In [225]:
def clean_time(df, col):
    df[col].replace('：',':',regex=True,inplace=True)
    df[col].replace('\.', ':', regex=True, inplace=True)
    df[col].replace(' ', '', regex=True, inplace=True)
    df[col].replace('点', ':', regex=True, inplace=True)
    df[col].replace('分', '', regex=True, inplace=True)
    df[col] = df[col].map(lambda x: datetime.strptime(x.strip(),'%H') if len(x)==1 else datetime.strptime(x.strip(),'%H:%M'))
    df[col] = df[col].dt.time

In [226]:
clean_time(df, 'arr_time')
clean_time(df, 'dep_time')

In [227]:
# remove space and change lower case to upper case
df['flight_num'] = df['flight_num'].map(lambda x: x.upper().replace(' ',''))
# remove none alphabet or none number
df['flight_num'] = df['flight_num'].map(lambda x: re.sub('[^0-9a-zA-Z]+', '', x))
# remove 0 that immediately positions after character KE0401 -> KE401
df['flight_num'] = df['flight_num'].map(lambda x: x[:2] + x[3:] if re.findall('[^a-zA-Z]', x)[0]=='0' else x)
# remove any characters and digtis after digits KE401A23 -> KE401
df['flight_num'] = df['flight_num'].map(lambda x: re.findall('(\D{2}\d{2,4}\D)', x)[0][:-1] if re.findall('(\D{2}\d{2,4}\D)', x) else x)
# fix edge cases
df['flight_num'].replace('KOREANAIR41', 'KE41', inplace=True)
df['flight_num'].replace('QZ272', 'OZ272', inplace=True)

In [228]:
df.to_csv('flight_info_clean.csv', index=False, encoding='utf-8-sig')

In [229]:
df.groupby(['arr_date', 'arr_time', 'flight_num']).count()['name']

arr_date  arr_time  flight_num
9月19日     10:30:00  KE401          1
                    KE41           8
          10:35:00  NH118          3
          13:25:00  EK229          1
          13:34:00  UA2315         1
          13:50:00  DL196          3
          16:20:00  TK203          1
          19:00:00  DL2250         1
9月20日     10:30:00  KE41           7
          10:35:00  NH118          3
          12:40:00  OZ272          1
          12:45:00  OZ272         41
                    OZ332          2
          12:55:00  OZ272          1
          17:25:00  NH118          1
          18:15:00  OZ272          1
9月21日     08:45:00  SQ28           2
          10:35:00  AA8446         1
          12:45:00  OZ272          3
          13:50:00  DL196          1
          14:20:00  DL282          1
Name: name, dtype: int64

In [230]:
# visualize the arrive time by count
df.groupby(['arr_date', 'arr_time']).count()['name']

arr_date  arr_time
9月19日     10:30:00     9
          10:35:00     3
          13:25:00     1
          13:34:00     1
          13:50:00     3
          16:20:00     1
          19:00:00     1
9月20日     10:30:00     7
          10:35:00     3
          12:40:00     1
          12:45:00    43
          12:55:00     1
          17:25:00     1
          18:15:00     1
9月21日     08:45:00     2
          10:35:00     1
          12:45:00     3
          13:50:00     1
          14:20:00     1
Name: name, dtype: int64

In [231]:
df.groupby(['arr_date', 'flight_num']).count()['name']

arr_date  flight_num
9月19日     DL196          3
          DL2250         1
          EK229          1
          KE401          1
          KE41           8
          NH118          3
          TK203          1
          UA2315         1
9月20日     KE41           7
          NH118          4
          OZ272         44
          OZ332          2
9月21日     AA8446         1
          DL196          1
          DL282          1
          OZ272          3
          SQ28           2
Name: name, dtype: int64